In [0]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from statistics import mean
import random
from pandas import DataFrame
import matplotlib
import numpy as np
from timeit import default_timer as timer
# be able to save images on server
matplotlib.use('Agg')
from matplotlib import pyplot
%matplotlib inline

top_words = 5000
(x_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [0]:
embedding_dimension = 50
top_words = 5000

def get_data(maxlen):
    x = sequence.pad_sequences(x_train, maxlen=maxlen,padding='post',truncating='post')
    return x,y_train

In [15]:
import itertools

#learning_rate = [0.01, 0.001, 0.05, 0.02]
#n_neurons = [32, 64, 128, 256]
#n_timesteps = [200, 300, 400, 500]
#n_epochs = [3, 6, 9, 12]

# random [0.018157239717328855, 464, 458, 5]
# hyper 0.0031  278  7  422  0.9350
# genetic type-1 best_model-[24, 54, 3, 429]
#genetic-type-2 # 0.017 & 112 & 4 & 418 & 0.938
#genetic-type-3 [15, 85, 3, 475]


learning_rate = [0.01815,0.0031,0.012,0.0075]
n_neurons = [464,278,54,85]
n_timesteps = [458,422,429,475]
n_epochs = [5,7,3]


hyper_para = (learning_rate, n_neurons, n_timesteps, n_epochs)
hyper_para_grid = tuple(itertools.product(*hyper_para, repeat=1))
print(len(hyper_para_grid))
for learning_rate, n_neurons, n_timesteps, n_epochs in hyper_para_grid:
  print('lr',learning_rate, 'n_neurons', n_neurons, 'n_timesteps', n_timesteps,'n_epochs', n_epochs)

192
lr 0.01815 n_neurons 464 n_timesteps 458 n_epochs 5
lr 0.01815 n_neurons 464 n_timesteps 458 n_epochs 7
lr 0.01815 n_neurons 464 n_timesteps 458 n_epochs 3
lr 0.01815 n_neurons 464 n_timesteps 422 n_epochs 5
lr 0.01815 n_neurons 464 n_timesteps 422 n_epochs 7
lr 0.01815 n_neurons 464 n_timesteps 422 n_epochs 3
lr 0.01815 n_neurons 464 n_timesteps 429 n_epochs 5
lr 0.01815 n_neurons 464 n_timesteps 429 n_epochs 7
lr 0.01815 n_neurons 464 n_timesteps 429 n_epochs 3
lr 0.01815 n_neurons 464 n_timesteps 475 n_epochs 5
lr 0.01815 n_neurons 464 n_timesteps 475 n_epochs 7
lr 0.01815 n_neurons 464 n_timesteps 475 n_epochs 3
lr 0.01815 n_neurons 278 n_timesteps 458 n_epochs 5
lr 0.01815 n_neurons 278 n_timesteps 458 n_epochs 7
lr 0.01815 n_neurons 278 n_timesteps 458 n_epochs 3
lr 0.01815 n_neurons 278 n_timesteps 422 n_epochs 5
lr 0.01815 n_neurons 278 n_timesteps 422 n_epochs 7
lr 0.01815 n_neurons 278 n_timesteps 422 n_epochs 3
lr 0.01815 n_neurons 278 n_timesteps 429 n_epochs 5
lr 0.018

In [0]:
train_loss_grid, val_loss_grid = list(), list()
best_params = []
def fit_lstm_grid(batch_size):
  min_val_loss = 999
  current_val_loss = 0
  for learning_rate, n_neurons, n_timesteps, n_epochs in hyper_para_grid:
    print('lr',learning_rate, 'n_neurons', n_neurons, 'n_timesteps', n_timesteps,'n_epochs', n_epochs)
    model = Sequential()
    model.add(Embedding(top_words, embedding_dimension,input_length=n_timesteps,mask_zero=True))
    model.add(LSTM(n_neurons,return_sequences=False))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=Adam(lr=learning_rate),metrics=['accuracy'])
    # fit model
    get_train_x,label_array = get_data(n_timesteps)
    lstm_model = model.fit(get_train_x,label_array,epochs=n_epochs,batch_size=batch_size, validation_split=0.2)
    current_val_loss = lstm_model.history['val_loss'][-1]
    if current_val_loss < min_val_loss:
      min_val_loss = current_val_loss
      best_params = [learning_rate,n_neurons,n_timesteps,n_epochs]
      print('best_params',best_params)
    
  return best_params


In [0]:
def run_lstm_grid():
  n_batch = 100
  best_params = fit_lstm_grid(n_batch)
  return best_params

In [19]:
start = timer()
best_params = run_lstm_grid()
end=timer()
print("Time Taken -> ",str(end-start))
print("Best params ",best_params)

lr 0.01815 n_neurons 464 n_timesteps 458 n_epochs 5
Epoch 1/5
200/200 [==============================] - 34s 172ms/step - loss: 0.7030 - accuracy: 0.5346 - val_loss: 0.5163 - val_accuracy: 0.7498
Epoch 2/5
200/200 [==============================] - 34s 168ms/step - loss: 0.4058 - accuracy: 0.8180 - val_loss: 0.3765 - val_accuracy: 0.8342
Epoch 3/5
200/200 [==============================] - 33s 166ms/step - loss: 0.3219 - accuracy: 0.8619 - val_loss: 0.3586 - val_accuracy: 0.8432
Epoch 4/5
200/200 [==============================] - 33s 165ms/step - loss: 0.2846 - accuracy: 0.8813 - val_loss: 0.3688 - val_accuracy: 0.8448
Epoch 5/5
200/200 [==============================] - 33s 166ms/step - loss: 0.2568 - accuracy: 0.8929 - val_loss: 0.3753 - val_accuracy: 0.8412
best_params [0.01815, 464, 458, 5]
lr 0.01815 n_neurons 464 n_timesteps 458 n_epochs 7
Epoch 1/7
200/200 [==============================] - 34s 169ms/step - loss: 0.8032 - accuracy: 0.4976 - val_loss: 0.7004 - val_accuracy: 0.51